# Introduction

Much like Agent 007's chances of dying as he's tied to a table with a spinning blade slowly but surely making its way towards his crotch, our 'random' agent has little to no chance of connecting 4 against a smart agent or human player. But it's a good place to start. In this notebook, I'll create a few agents and then use them as benchmarks against smarter and more complex agents to see if I can build an agent that's actually capable fo beating a human, or at least acheiving a good place on the leaderboard.

In [ ]:
import random
import numpy as np

from learntools.core import binder
binder.bind(globals())
from learntools.game_ai.ex1 import *

# The ConnectX environment

Let's load in the ConnectX environment from kaggle_environments, take a little look at it and make two agents play against each other. The code below also shows two 'pre-build' agents available in the environment - random and negamax.

In [ ]:
from kaggle_environments import make, evaluate, utils
env = make("connectx", debug=True)
print(list(env.agents))

We can make two agents play each other with the code.

In [ ]:
env.run(["random", "random"])
env.render(mode="ipython")

# Helper Functions

In [ ]:
# This function takes two agents, and makes them play 100 games against each other
# Each agent gets 50 games where it gets to make the first move
# It calculates the percentage of games won by agent1 and agent2

def get_win_percentages(agent1, agent2, n_rounds):
    # Use default Connect Four setup
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # Agent 1 goes first (roughly) half the time          
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    # Agent 2 goes first (roughly) half the time      
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    print("Agent 1 Win Percentage:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))

In [ ]:
# This file will write our agent functions to output

import inspect
import os

def write_agent_to_file(function, file):
    with open(file, "a" if os.path.exists(file) else "w") as f:
        f.write(inspect.getsource(function))
        print(function, "written to", file)

# Agent Random

Next, let's define a random agent, similar to the pre-build one that comes with ConnectX. All this agent does is create a list of valid moves i.e which columns is it possible to put a marker in. From this list of valid columns, it chooses one at random.

In [ ]:
# Selects random valid column
def agent_random(obs, config):
    valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    return random.choice(valid_moves)

# Agent Smart

The first challenge is to build an agent using some simple rules that can beat Agent Random. A good place to start is an agent that takes a winning move if it's available and blocks the winning move of an opponent if it's available. And when no winning moves are available, just plays randomly.

For submission purposes, the function for an agent has to be fully self-contained. So inside the agent_smart function, we'll define a couple of sub-functions. The first, drop_piece, builds a view of what the board will look like if we drop a marker into a certain column. The second, check_winning_move, will check to see if dropping a marker in a certain column will win the game, either for our agent or it's opponent.

In [ ]:
def agent_smart(obs, config):
    
    ################################
    # Imports and helper functions #
    ################################
    
    import numpy as np
    import random

    # Gets board at next step if agent drops piece in selected column
    def drop_piece(grid, col, piece, config):
        next_grid = grid.copy()
        for row in range(config.rows-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = piece
        return next_grid

    # Returns True if dropping piece in column results in game win
    def check_winning_move(obs, config, col, piece):
        # Convert the board to a 2D grid
        grid = np.asarray(obs.board).reshape(config.rows, config.columns)
        next_grid = drop_piece(grid, col, piece, config)
        # horizontal
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(next_grid[row,col:col+config.inarow])
                if window.count(piece) == config.inarow:
                    return True
        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(next_grid[row:row+config.inarow,col])
                if window.count(piece) == config.inarow:
                    return True
        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = list(next_grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                if window.count(piece) == config.inarow:
                    return True
        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(next_grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                if window.count(piece) == config.inarow:
                    return True
        return False
    
    #########################
    # Agent makes selection #
    #########################
    
    valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    for col in valid_moves:
        if check_winning_move(obs, config, col, obs.mark):
            return col
    for col in valid_moves:
        if check_winning_move(obs, config, col, obs.mark%2+1):
            return col
    return random.choice(valid_moves)

Let's see agent_smart play against agent_random.

In [ ]:
# Agents play one game round
env.run([agent_smart, agent_random])

# Show the game
env.render(mode="ipython")

Now let's see how many times agent_smart beats agent_random in 100 games.

In [ ]:
get_win_percentages(agent1=agent_smart, agent2=agent_random, n_rounds=100)

Agent smart wins 96 games out of 100 against agent_random. That's good, now agent_smart will be our bechmark for even better agents.

There are lots more complicated rules we could come up with for agent_smart to make it even better. For example, we might tell it to look to make a connect 3 if it's available or block the connect 3 of an opponent. Well, actually that makes the agent worse. It looks to build connect 3s that can never become connect 4s and so wastes turns. We need an agent that's capable of looking ahead of the move that it's making and instead of making lots of explicit rules, let's use heuristics to let our agent know what kind of moves and situations will be to it's advantage.

# Agent_lookahead

Next we'll define an agent that looks one move ahead, at every possible move and assigns a score (heuristic) to each possible new configuration of the game board. It will then take the move that gives the best score. The three scenarios we will give scores to are

* **A:** The agent has four discs in a row (the agent won),
* **B:** The agent filled three spots, and the remaining spot is empty (the agent wins if it fills in the empty spot).
* **C:** The agent filled two spots, and the remaining two spots are empty (the agent wins if it fills in the empty two spots).
* **D:** The opponent filled two spots, and the remaining two spots are empty (the opponent wins by filling in the empty two spots).
* **E:** The opponent filled three spots, and the remaining spot is empty (the opponent wins by filling in the empty spot).

In [ ]:
def agent_lookahead(obs, config):
    
    ################################
    # Imports and helper functions #
    ################################
    
    import numpy as np
    import random
    
    A = 1e10
    B = 1e4
    C = 1e2
    D = -1
    E = -1e6
    
    # Calculates score if agent drops piece in selected column
    def score_move(grid, col, mark, config):
        next_grid = drop_piece(grid, col, mark, config)
        score = get_heuristic(next_grid, col, mark, config)
        return score

    # Helper function for score_move: gets board at next step if agent drops piece in selected column
    def drop_piece(grid, col, mark, config):
        next_grid = grid.copy()
        for row in range(config.rows-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = mark
        return next_grid

    # Helper function for score_move: calculates value of heuristic for grid    
    def get_heuristic(grid, col, mark, config):
        num_twos = count_windows(grid, 2, mark, config)
        num_threes = count_windows(grid, 3, mark, config)
        num_fours = count_windows(grid, 4, mark, config)
        num_twos_opp = count_windows(grid, 2, mark%2+1, config)
        num_threes_opp = count_windows(grid, 3, mark%2+1, config)
        score = A*num_fours + B*num_threes + C*num_twos + D*num_twos_opp + E*num_threes_opp
        return score

    # Helper function for get_heuristic: checks if window satisfies heuristic conditions
    def check_window(window, num_discs, piece, config):
        return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)
    
    # Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
    def count_windows(grid, num_discs, piece, config):
        num_windows = 0
        # horizontal
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[row, col:col+config.inarow])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(grid[row:row+config.inarow, col])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        return num_windows
    
    #########################
    # Agent makes selection #
    #########################
    
    # Get list of valid moves
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    # Use the heuristic to assign a score to each possible board in the next turn
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config) for col in valid_moves]))
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    # Select at random from the maximizing columns
    return random.choice(max_cols)

In [ ]:
# Agents play one game round
env.run([agent_lookahead, agent_smart])

# Show the game
env.render(mode="ipython")

In [ ]:
get_win_percentages(agent1=agent_lookahead, agent2=agent_smart, n_rounds=100)

# Agent_minimax

Lookahead playes reasonably well but if you watch a few of it's games, you start to see patterns of weakness - it will often give a smart opponent an easy opportunity to fill in the gap in a connect 4. It does this because it doesn't look further ahead than the next move. We need an agent that can look more than one move ahead.

The minimax algorithm looks multiple steps ahead and builds pictures of potential gameboards and scores them with heuristics. It assumes that the opponent will then make the optimal move, based on the heuristic. Our agent gets to make another move and it again evaulates the next gameboard. This can go on, based on how many steps we want to look ahead.

It's called minimax because our agent wants to maximise the score and we assume our opponent will try to minimize it. The agent will make decisions based on getting the maximum possible score.

In [ ]:
def agent_minimax(obs, config):
    
    ################################
    # Imports and helper functions #
    ################################
    
    import numpy as np
    import random
    
    # How deep to make the game tree: higher values take longer to run!
    N_STEPS = 3
    
    A = 1e10
    B = 1e4
    C = 1e2
    D = -1
    E = -1e6
    
    # Gets board at next step if agent drops piece in selected column
    def drop_piece(grid, col, mark, config):
        next_grid = grid.copy()
        for row in range(config.rows-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = mark
        return next_grid

    # Helper function for get_heuristic: checks if window satisfies heuristic conditions
    def check_window(window, num_discs, piece, config):
        return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)
    
    # Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
    def count_windows(grid, num_discs, piece, config):
        num_windows = 0
        # horizontal
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[row, col:col+config.inarow])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(grid[row:row+config.inarow, col])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        return num_windows
    
    # Helper function for score_move: calculates value of heuristic for grid    
    def get_heuristic(grid, mark, config):
        num_twos = count_windows(grid, 2, mark, config)
        num_threes = count_windows(grid, 3, mark, config)
        num_fours = count_windows(grid, 4, mark, config)
        num_twos_opp = count_windows(grid, 2, mark%2+1, config)
        num_threes_opp = count_windows(grid, 3, mark%2+1, config)
        score = A*num_fours + B*num_threes + C*num_twos + D*num_twos_opp + E*num_threes_opp
        return score
    
    # Uses minimax to calculate value of dropping piece in selected column
    def score_move(grid, col, mark, config, nsteps):
        next_grid = drop_piece(grid, col, mark, config)
        score = minimax(next_grid, nsteps-1, False, mark, config)
        return score
    
    # Helper function for minimax: checks if agent or opponent has four in a row in the window
    def is_terminal_window(window, config):
        return window.count(1) == config.inarow or window.count(2) == config.inarow
    
    # Helper function for minimax: checks if game has ended
    def is_terminal_node(grid, config):
        # Check for draw 
        if list(grid[0, :]).count(0) == 0:
            return True
        # Check for win: horizontal, vertical, or diagonal
        # horizontal 
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[row, col:col+config.inarow])
                if is_terminal_window(window, config):
                    return True
        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(grid[row:row+config.inarow, col])
                if is_terminal_window(window, config):
                    return True
        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                if is_terminal_window(window, config):
                    return True
        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                if is_terminal_window(window, config):
                    return True
        return False

    # Minimax implementation
    def minimax(node, depth, maximizingPlayer, mark, config):
        is_terminal = is_terminal_node(node, config)
        valid_moves = [c for c in range(config.columns) if node[0][c] == 0]
        if depth == 0 or is_terminal:
            return get_heuristic(node, mark, config)
        if maximizingPlayer:
            value = -np.Inf
            for col in valid_moves:
                child = drop_piece(node, col, mark, config)
                value = max(value, minimax(child, depth-1, False, mark, config))
            return value
        else:
            value = np.Inf
            for col in valid_moves:
                child = drop_piece(node, col, mark%2+1, config)
                value = min(value, minimax(child, depth-1, True, mark, config))
            return value
    
    #########################
    # Agent makes selection #
    #########################
    
    # If agent gets first move, put marker in middle column
    #if sum(obs.board) == 0:
        #return 3
    # Get list of valid moves
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    # Use the heuristic to assign a score to each possible board in the next step
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, N_STEPS) for col in valid_moves]))
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    # Select at random from the maximizing columns
    return random.choice(max_cols)

I've found that using agent_minimax seems to work well but fails when I try to submit it. This is because there's a rule in the game where your agent is disqualified if it takes longer than 5 seconds to make a move.

So, I'll use a minimax alphabeta agent instead, which is a little quicker. And, importantly, it only seems to be able to satisfy the 5 second rule if N is 2. Once you go to 3 or over, it takes too long.

In [ ]:
def agent_alphabeta(obs, config):

    ################################
    # Imports and helper functions #
    ################################
    
    import numpy as np
    import random

    # Helper function for is_terminal_node: checks if agent or opponent has four in a row in the window
    def is_terminal_window(window, config):
        return window.count(1) == config.inarow or window.count(2) == config.inarow

    # Helper function for minimax_alphabeta: checks if game has ended
    def is_terminal_node(grid, config):
        # Check for draw
        # The list method would be faster.
        #valid_moves = [c for c in range(config.columns) if grid[0][c] == 0]
        #if len(valid_moves) == 0:
        if list(grid[0, :]).count(0) == 0:
            return True
        # Check for win: horizontal, vertical, or diagonal
        # horizontal 
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[row, col:col+config.inarow])
                if is_terminal_window(window, config):
                    return True
        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(grid[row:row+config.inarow, col])
                if is_terminal_window(window, config):
                    return True
        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                if is_terminal_window(window, config):
                    return True
        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                if is_terminal_window(window, config):
                    return True
        return False

    # Helper function for count_windows: checks if window satisfies heuristic conditions
    def check_window(window, num_discs, piece, config):
        return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)

    # Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
    def count_windows(grid, num_discs, piece, config):
        num_windows = 0
        # horizontal
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[row, col:col+config.inarow])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(grid[row:row+config.inarow, col])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        return num_windows

    # Helper function for minimax: calculates value of heuristic for grid
    def get_heuristic(grid, mark, config):
        num_threes = count_windows(grid, 3, mark, config)
        num_fours = count_windows(grid, 4, mark, config)
        num_threes_opp = count_windows(grid, 3, mark%2+1, config)
        num_fours_opp = count_windows(grid, 4, mark%2+1, config)
        return num_threes - 1e2*num_threes_opp - 1e4*num_fours_opp + 1e6*num_fours

    # Gets board at next step if agent drops piece in selected column
    def drop_piece(grid, col, mark, config):
        next_grid = grid.copy()
        for row in range(config.rows-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = mark
        return next_grid

    # Minimax implementation: recursive!!!
    # https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning#Pseudocode
    #def minimax_alphabeta(node, depth, alpha, beta, maximizingPlayer, mark, config):
    def minimax_alphabeta(node, depth, alpha, beta, maximizingPlayer, mark, config, columns_centered): # columns_centered
        if depth == 0 or is_terminal_node(node, config):
            return get_heuristic(node, mark, config)
        #valid_moves = [c for c in range(config.columns) if node[0][c] == 0]
        valid_moves = [c for c in columns_centered if node[0][c] == 0] # columns_centered
        if maximizingPlayer:
            value = -np.Inf
            for col in valid_moves:
                child = drop_piece(node, col, mark, config)
                #value = max(value, minimax_alphabeta(child, depth-1, alpha, beta, False, mark, config))
                value = max(value, minimax_alphabeta(child, depth-1, alpha, beta, False, mark, config, columns_centered)) # columns_centered
                alpha = max(alpha, value) # the minimum score that the maximizing player (i.e., the "alpha" player) is assured of
                if alpha >= beta: #
                    break # beta cutoff
            return value
        else:
            value = np.Inf
            for col in valid_moves:
                child = drop_piece(node, col, mark%2+1, config)
                #value = min(value, minimax_alphabeta(child, depth-1, alpha, beta, True, mark, config))
                value = min(value, minimax_alphabeta(child, depth-1, alpha, beta, True, mark, config, columns_centered)) # columns_centered
                beta = min(beta, value) # the maximum score that the minimizing player (i.e. the "beta" player) is assured of
                if beta <= alpha: #
                    break # alpha cutoff
            return value

    # Uses minimax to calculate value of dropping piece in selected column
    #def score_move(grid, col, mark, config, nsteps):
    def score_move(grid, col, mark, config, nsteps, columns_centered): # columns_centered
        #return minimax_alphabeta(grid, nsteps, -np.Inf, np.Inf, True, mark, config)        # no need to create next_grid here. minimax_alphabeta() will do that.
        # but this method of calculating next_grid is faster!
        next_grid = drop_piece(grid, col, mark, config)
        #return minimax_alphabeta(next_grid, nsteps-1, -np.Inf, np.Inf, False, mark, config)
        return minimax_alphabeta(next_grid, nsteps-1, -np.Inf, np.Inf, False, mark, config, columns_centered) # columns_centered


    #########################
    # Agent makes selection #
    #########################

    # How deep to make the game tree: higher values take longer to run!
    N_STEPS = 2

    # Get list of valid moves
    #valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    # columns more center would get higher heuristics. This could be useful in pruning.
    #columns = [c for c in range(config.columns)] # error in v17 why?
    columns = [c for c in range(config.columns) if obs.board[c] == 0]
    dist_from_center = {c: abs(c-(config.columns-1)/2) for c in columns}
    columns_centered = [k for k, v in sorted(dist_from_center.items(), key=lambda item: item[1])]
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    # Use the heuristic to assign a score to each possible board in the next step
    #scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, N_STEPS) for col in valid_moves]))
    scores = dict(zip(columns_centered, [score_move(grid, col, obs.mark, config, N_STEPS, columns_centered) for col in columns_centered])) # columns_centered
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    # Select at random from the maximizing columns
    return random.choice(max_cols)

In [ ]:
# Agents play one game round
env.run([agent_alphabeta, agent_alphabeta])

# Show the game
env.render(mode="ipython")

In [ ]:
get_win_percentages(agent1=agent_alphabeta, agent2=agent_lookahead, n_rounds=2)

In [ ]:
write_agent_to_file(agent_alphabeta, "submission.py")

In [ ]:
import sys
out = sys.stdout
submission = utils.read_file("/kaggle/working/submission.py")
agent = utils.get_last_callable(submission)
sys.stdout = out

env = make("connectx", debug=True)
env.run([agent_alphabeta, agent_alphabeta])
print("Success!" if env.state[0].status == env.state[1].status == "DONE" else "Failed...")

In [ ]:
env.play([None, agent_minimax], width=500, height=450)